In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip

In [ ]:
!unzip "BelgiumTSC_Training.zip"

In [ ]:
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip

In [ ]:
!unzip "BelgiumTSC_Testing.zip"

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

In [ ]:
# Define the CNN model
def create_model(input_shape, num_classes):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model


In [ ]:
# Define dataset paths and parameters
train_data_dir = 'Training'
valid_data_dir = 'Testing'
input_shape = (224, 224, 3)  # Adjust based on your image dimensions
num_classes = 62  # Number of road sign classes

In [ ]:
# Create the model
model = create_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data preprocessing and augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical'
)

Found 4575 images belonging to 62 classes.
Found 2520 images belonging to 62 classes.


In [ ]:
# Train the model
epochs = 10  # Adjust based on your dataset and computational resources
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)

Epoch 1/10
143/143 [==============================] - 85s 597ms/step - loss: 0.2232 - accuracy: 0.9305 - val_loss: 0.2325 - val_accuracy: 0.9413
Epoch 2/10
143/143 [==============================] - 85s 595ms/step - loss: 0.1989 - accuracy: 0.9364 - val_loss: 0.3029 - val_accuracy: 0.9290
Epoch 3/10
143/143 [==============================] - 86s 601ms/step - loss: 0.1922 - accuracy: 0.9366 - val_loss: 0.2631 - val_accuracy: 0.9365
Epoch 4/10
143/143 [==============================] - 85s 594ms/step - loss: 0.1660 - accuracy: 0.9478 - val_loss: 0.2268 - val_accuracy: 0.9437
Epoch 5/10
143/143 [==============================] - 85s 596ms/step - loss: 0.1585 - accuracy: 0.9504 - val_loss: 0.2424 - val_accuracy: 0.9393
Epoch 6/10
143/143 [==============================] - 86s 597ms/step - loss: 0.3249 - accuracy: 0.9121 - val_loss: 0.2402 - val_accuracy: 0.9484
Epoch 7/10
143/143 [==============================] - 85s 594ms/step - loss: 0.1532 - accuracy: 0.9495 - val_loss: 0.2635 - val_ac

In [ ]:
# Save the model
model.save('road_sign_detection_model.h5')